In [ ]:
%matplotlib inline

# Assignment 3

**DUE: Tuesday, February 20th 2023 at 5:00pm**

Turn in the assignment via Canvas with the following *three* parts:
1.  The ipynb.
2.  PDF reports including their ipynb screenshots
3.  A survey

  a.  Create a new copy in google drive. Complete the survey using the provided drop down menu, and submit a csv file or google sheet link.

  b.  https://docs.google.com/spreadsheets/d/1FijZmoQEsOM7NXk50r-yMF6OLU9EbuGUd8YFcfIR-00/edit?usp=sharing

To write legible answers you will need to be familiar with both [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) and [Latex](https://www.latex-tutorial.com/tutorials/amsmath/)

Before you turn this problem in, make sure everything runs as expected. To do so, restart the kernel and run all cells (in the menubar, select Runtime→→Restart and run all).
#### Show your work!
Whenever you are asked to find the solution to a problem, be sure to also **show how you arrived** at your answer.

### Resources

[1] https://community.plotly.com/t/two-3d-surface-with-different-color-map/18931/2 (Different color plots plotly)

[2] https://plotly.com/python/3d-surface-plots/ [plotting surfaces]

[3] https://community.plotly.com/t/3d-scatter-plot-with-surface-plot/27556/5 [plotting scatter plots]

[4] https://community.plotly.com/t/what-colorscales-are-available-in-plotly-and-which-are-the-default/2079 [color scales]

In [ ]:
%matplotlib inline

In [ ]:
NAME = "Shayan Salsabilian"
STUDENT_ID = "1585653"

## Problem 1: Local Search on the Ackley Surface
---

As discussed in class with Local Search problems the path to the goal is not as important as the goal itself. In this question we work with the Ackley function, a non-convex function typically used as a test for optimization algorithms.

The Achkey function has many local optima but only one global optimum that is $f(0,0) = 0$. Interestingly enough it might not be that easy to find this solution.

The Ackley function is defined as:

$$f(x,y):= -20exp[-0.2\sqrt{0.5(x^2 + y^2)}] -exp[0.5(cos(2\pi x) + cos(2\pi y))] + e + 20$$

The figure below illustrates the result of running three optimization methods on the Ackley Function. These solutions were found using methods A, B and C.


![](https://drive.google.com/uc?id=1K0_iokZu-TgjD0xrfGmWQbb-r67kINI0)


In this question, you will implement 3 optimization algorithms: 

1.   Stochastic Hill Climbing with Restarts (SHCR)
2.   Simulated Annealing (SA)
3.   Local Beam Search (LBS)

**Deliverables**

1. Complete this Notebook with implementation of SHCR, SA and LBS.
2. For each algorithm **visualize** the candidate solutions found at each
step using the provided ```create_ackley_figure()``` function. please see the image above an example of an image that should be included in your submission.



In [ ]:
import numpy as np
from numpy import arange
from numpy import exp
from numpy import sqrt
from numpy import cos
from numpy import e
from numpy import pi
from numpy import meshgrid
from numpy.random import randn
from numpy.random import rand
from numpy.random import seed
# hill climbing search of the ackley objective function
from numpy import asarray
import plotly.graph_objects as go

# objective function
def f(x, y):
	return -20.0 * exp(-0.2 * sqrt(0.5 * (x**2 + y**2))) - exp(0.5 * (cos(2 * pi * x) + cos(2 * pi * y))) + e + 20

def create_ackley_figure(solution_points=np.array([]), soln_names=['A','B,','C']):
  """
  :param solution_points A numpy array of dimensions [i, n, 2]
  where i is the number of solution points you want to plot and n is the 
  number of steps used in estimating your solution. In each case use the same 
  number of steps if you want to plot all solutions on the same figure.

  :return An interactive Ackley function figure with/without solution points.
  """
  # define range for input
  r_min, r_max = -5.0, 5.0
  # sample input range uniformly at 0.1 increments
  xaxis = arange(r_min, r_max, 0.1) # (100,)
  yaxis = arange(r_min, r_max, 0.1) # (100,)
  # create a mesh from the axis
  x, y = meshgrid(xaxis, yaxis)

  # compute targets
  results = f(x, y)

  figure = go.Figure(data=[go.Surface(z=results, x=x, y=y,colorscale='Blues',showscale=False)])
  figure.update_layout(title='Ackley Function', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
  
  if solution_points.size:
    soln_colors = ['Reds', 'Rainbow', 'Viridis']
    for i in range(len(solution_points)):
      x_sln = solution_points[i][:,0]
      y_sln = solution_points[i][:,1]
      zdata = f(x_sln, y_sln)
      figure.add_scatter3d(name=soln_names[i], x=x_sln, y=y_sln, z=zdata, mode='markers',
      marker=dict(size=1, color=x_sln.flatten(), colorscale=soln_colors[i]))
      
      figure.update_layout(showlegend=True)
      
  return figure

In [ ]:
### RUN THIS ###
"""
This is the Ackley Function.
"""
ack_figure = create_ackley_figure()
ack_figure

### General Function Definitions

In [ ]:
# objective function
def objective(v):
  """
  This function defines the Ackley surface, it has a minimum of 0. This 
  can be used to test if we have found points that yield the minimum value.
  :param v, a tuple representing a 2D point being tested.

  returns a value in the range [-5.0, 5.0]
  """
  x, y = v
  return -20.0 * exp(-0.2 * sqrt(0.5 * (x**2 + y**2))) - exp(0.5 * (cos(2 * pi * x) + cos(2 * pi * y))) + e + 20

  
# check if a point is within the bounds of the search
def in_bounds(point, bounds=asarray([[-5.0, 5.0], [-5.0, 5.0]])):
  """
  It is possible our optimzation method could move us outside of our search 
  space, so it is helpful to check.
  :param point a tuple representing a 2D point
  :param a 2D array, that describes the bounds of each dimension of our point.
  returns a Boolean of whether the point lies in the bounds or not.
  """
	# enumerate all dimensions of the point
  for d in range(len(bounds)):
    # check if out of bounds for this dimension
    if point[d] < bounds[d, 0] or point[d] > bounds[d, 1]:
      return False
  return True

### Part 1) Stochastic Hill Climbing with Restarts

1.   Implement SHCR in the cell below.
2.   Visualize and comment on the candidate points visited.


In [ ]:
# stochastic hill climbing with restarts algorithm
def stochastic_hillclimbing(objective, bounds, n_iterations, step_size, start_pt):
  """
  :param objective, the function we are trying to optimize
  :param bounds, the boundaries of the problem
  :param n_iterations number of times to repeat stochastic hill climbing
  :param step_size how much we should move
  :param start_pt the point we start optimizing from.
  returns [solution, solution_value, candidates]
  """
  ### YOUR CODE HERE ###
  solution_value = objective(start_pt)
  solution = start_pt # check how good the start point is
  candidates = [] # keep track of potential points closest to the maxima
  for i in range(n_iterations): # have a max amount of iterations
    candidate = None
    while candidate is None or not in_bounds(candidate, bounds): # make sure we have a viable point
      candidate = solution + randn(len(bounds))*step_size # take a step from the point
      candidates.append([candidate[0], candidate[1]])
      cand_val =  objective(candidate) # score it to see if its better then our original
    if (cand_val <= solution_value):
      solution_value = cand_val # if it is thats our new potential point closest to the maxima
      solution = candidate
  return [solution, solution_value, candidates]

def random_restarts(objective, bounds, n_iter, step_size, n_restarts):
  """
  :param objective, the function we are trying to optimize
  :param bounds, the boundaries of the problem
  :param n_iter number of times to repeat stochastic hill climbing 
  :param step_size how much we should move
  :param n_restarts, the number of times we restart.
  returns [best solution, best solution value, visited points]
  """
  ### YOUR CODE HERE ###
  best_solution, best_solution_value, visited_points = None, np.inf, []
  visited_points = []
  for n in range(n_restarts): # run for the max number of restarts
    while(1):
      start_pt = bounds[:, 0] + rand(len(bounds)) * (bounds[:,1] - bounds[:,0]) # get a new random point for the restart
      if start_pt is not None and in_bounds(start_pt, bounds): # if its not viable keep looking
        break
    solution, solution_value, candidates = stochastic_hillclimbing(objective, bounds,n_iter, step_size, start_pt) # other wise run the hillclimbing algorithm on our 
    if solution_value < best_solution_value: # generated point
      best_solution = solution # if this point (our one of the steps near it) yielded a better maxima then our current best max replace it
      best_solution_value = solution_value
      visited_points.append(candidates)
  return [best_solution, best_solution_value, visited_points]

# seed the pseudorandom number generator
seed(240)
# define range for input
bounds = asarray([[-5.0, 5.0], [-5.0, 5.0]])
# define the total iterations
n_iter = 1000
# define the maximum step size
step_size = 0.05
# total number of random restarts
n_restarts = 100
# perform the hill climbing search
best, score, shcr_candidates = random_restarts(objective, bounds, n_iter, step_size, n_restarts)
print('Done!')
print('f(%s) = %f' % (best, score))

Done!
f([-0.00111338 -0.00190857]) = 0.006380


In [ ]:
# An example of plotting solutions from 3 optimziation methods.
# all_method_candidates has shape [3, 1000, 2], the first dim is the number of solutions
# the second dim is the number of steps taken in each method.
shcr_candidates = np.array(shcr_candidates)
shcr_candidates = shcr_candidates.reshape(-1,2)
all_method_candidates = np.array([shcr_candidates])
ack_figure = create_ackley_figure(all_method_candidates)
ack_figure

The candidate randomly restarts at multiple points throughout the ackley function, if we continued infinitelly, we would eventually hit the point 0,0 where the local optima is but even with 100 restarts we get pretty close at point [-0.00111338 -0.00190857].

### Part 2) Simulated Annealing

1.   Implement SA in the cell below.
2.   Visualize and comment on the candidate points visited.

In [ ]:
def get_temperature_schedule(epoch, temp, diff):
  """
  :param temp Temperature
  :param epoch Iterations elapsed
  :param diff Difference between value of candidate solution and value of best solution so far.
  """
  # calculate temperature for current epoch
  t = temp / float(epoch + 1)
  # calculate acceptance criterion
  criterion = exp(-diff / t)
  return criterion

# simulated annealing algorithm
def simulated_annealing(objective, bounds, n_iterations, step_size, temp):
  """ 
  :param objective, the function we are trying to optimize
  :param bounds, the boundaries of the problem
  :param n_iterations number of times to repeat stochastic hill climbing
  :param step_size how much we should move
  :param temp temperature for each epoch
  returns [solution, solution_value, candidates] 
  """
  ### YOUR CODE HERE ###
  solution = bounds[:, 0] + rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0]) # get a starting point
  solution_value = objective(solution) #check how good it is
  current = solution  #start the algorithm on this starting point
  current_value = solution_value
  candidates = []
  for i in range(n_iterations):
    candidate =  current + randn(len(bounds)) * step_size #take a step
    candidate_value = objective(candidate) #check how good it is
    candidates.append(candidate) #mark that we checked this point
    if candidate_value < solution_value: #check if it is better than our current best solution
      solution = candidate #save it if it is
      solution_value = candidate_value
    diff = candidate_value - current_value
    t = get_temperature_schedule(i, temp, diff) # get our temperature (system to determine if we should keep exploring or exploit)
    if diff < 0  or rand() < t: #if the value is negative (i.e. less than our current value) or we probabilistically are less than t
      current = candidate # check this point next
      current_value = candidate_value 
  return [solution, solution_value, candidates] #save our lowest points

# seed the pseudorandom number generator
seed(240)
# define range for input
bounds = asarray([[-5.0, 5.0], [-5.0, 5.0]])
# define the total iterations
n_iterations = 1000
# define the maximum step size
step_size = 0.1
# initial temperature
temp = 10
# perform the simulated annealing search
best, score, sa_candidates = simulated_annealing(objective, bounds, n_iterations, step_size, temp)
print('Done!')
print('f(%s) = %f' % (best, score))

Done!
f([-0.00300389  0.00018603]) = 0.008754


In [ ]:
sa_candidates = np.array(sa_candidates)
sa_candidates = sa_candidates.reshape(-1,2)
all_method_candidates = np.array([shcr_candidates,sa_candidates])
ack_figure = create_ackley_figure(all_method_candidates)
ack_figure

<ipython-input-14-f7ec91d8b44a>:3: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



Simulated Annealing will search its neighbors sometimes choosing those temporarily further away until it finds the global optima. This is why it is more concentrated than random restart. However, it was slightly worse than random restart finding a lowest point of [-0.00300389  0.00018603]

### Part 3) Local Beam Search



1.   Implement LBS in the cell below.
2.   Visualize and comment on the candidate points visited.
3.   Compare all 3 implementations by commenting on the distribution of points on the Ackley surface and the empirical run time of each method.


In [ ]:
# generate the neighbors based on step size
def generate_neighbors(point, step_size=0.1):
    point = point.reshape(-1,1)
    possible_steps = possible_steps = np.array([[step_size*i for i in range(-10,11)],
                           [step_size*i for i in range(-10,11)]])
    neighbors = point + possible_steps
    return neighbors

def local_beam_search(objective, bounds, step_size, k, n_iterations):
  """ 
  :param objective, the function we are trying to optimize
  :param bounds, the boundaries of the problem
  :param k, how many candidates to consider
  :param n_iterations, how long to search for.
  returns [solution, solution_value, candidates] 
  """
  ### YOUR CODE HERE ###
  points = []
  candidates = []
  solution = np.inf
  solution_value = -np.inf
  for i in range(k):
    point =  bounds[:, 0] + rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0]) #generate our random points
    point_sol = objective(point) #score how good our initial points are
    points.append([point, point_sol]) #put them in a list
  for i in range(n_iterations): 
    current = []
    for j in points: # go through our points
      neigh =  generate_neighbors(np.array(j[0])) #find its neighbors
      for y in range(len(neigh[0])):
        point = [neigh[0][y], neigh[1][y]] 
        point_value = objective(point) #score how good our neighbors points are
        current.append([point, point_value]) #save there score and point
        candidates.append(point) # save candidates that we visited
      current.sort(key=lambda x:x[1]) # sort our points by the highest score
      if(current[0][1] > solution_value): # check if the current best scoring point is better and if it is replace it
        solution = current[0][0]
        solution_value = current[0][1]
      current = current[:k] # save the k best highest scores as our next iteration of points and do this again
    points = current
  return [solution, solution_value, candidates]

# seed the pseudorandom number generator
seed(240)
# define range for input
bounds = asarray([[-5.0, 5.0], [-5.0, 5.0]])
# define the total iterations
n_iterations = 10
# define the maximum step size
step_size = 0.1
# candidates to consider
k = 10
# perform the local beam search
sequences, sequences_score, candidates = local_beam_search(objective, bounds, step_size, k, n_iterations)
print('Done!')
print('f(%s) = %f' % (sequences, sequences_score))

Done!
f([-1.0649345988487577, 0.3037686572138494]) = 4.276804


In [ ]:
candidates = np.array(candidates)
candidates = candidates.reshape(-1,2)
all_method_candidates = np.array([shcr_candidates,sa_candidates, candidates])
ack_figure = create_ackley_figure(all_method_candidates)
ack_figure

<ipython-input-35-49caac09d124>:3: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



The candidate points and there neighbors are randomely generated across the graph, which is why there far more sporiadic than the previous two hill climbing algorithms. Instead, we essentially see these lines generated at random places with the lines essentially representing there neighbors that we eventually visit.

All 3 climbing algorithms got reasonably close to point (0,0) which is the global optima, but local beam search was the furthest way at -1.0649345988487577, 0.3037686572138494 and random restart was the closest at [-0.00111338 -0.00190857]. However, local beam search has the lowest distribution of points, followed by simulated annealing, followed by random restart. Therefore, we can see an inverse relationship between points visited and how close we get to the local optima. Furthermore, Local beam search has the longest runtime as we are creating k points then creating n neighbors for each k point making a runtime of kn, while simulated annealing and random restart both have a O(n) runtime making them faster. However, simulated annealing will be faster than random restart because it doesnt have to wait for a start point thats in_bounds and not None. 

## Problem 2: CSP
---

Consider the following constraint satisfaction problem. A linear graph has nodes of the following colors:

- Red
- Yellow
- Green
- Blue
- Violet

Each node has a domain of {1, 2, ..., 9}.

Each node type has the following constraints on its value:

- Red - No contraints
- Yellow - equals the rightmost digit of of the product of all its neighbors
- Green - equals the rightmost digit of the sum of all its neighbors 
- Blue - equals the leftmost digit of the sum of all its neighbors
- Violet - equals the leftmost digit of the product of all of its neighbors

As a reminder here is the pseudo code for the Min-Conflicts search algorithm:

![minconflicts](https://images2017.cnblogs.com/blog/1126979/201712/1126979-20171224140802287-1871895433.png)

**Notes:**

- It's possible that you won't converge to a solution in a single run. Try a few runs to see if you get to a solution.
- The testcases are to show you what a problem looks like, we will test/grade your program on different examples.

### Part 1) Implementation
Complete the function ```solve_csp``` defined below. You may find some helper functions useful.

In [9]:
import numpy as np
def solve_csp(nodes, arcs, max_steps):
    """
    This function solves the csp using the MinConflicts Search Algorithm.

    :param nodes, a list of letters that indicates what type of node it is,
                  the index of the node in the list indicates its id
                  letters = {R, Y, G, B, V}
                
    :param arcs,  a list of tuples that contains two numbers, indicating the 
                  IDs of the nodes the arc connects. 
                
    :param max_steps, max number of steps to make before giving up

    returns a list of values for the solution to the CSP where the 
             index of the value corresponds the value for that given node.
    """
    node_values = []
    ### YOUR CODE HERE ###
    def pot_Conflicts(arcs): # create a dictionary of neighbor nodes
      conflicts = {}
      for arc in arcs:
        if arc[0] not in conflicts:
         conflicts[arc[0]] = [arc[1]]
        else:
          conflicts[arc[0]].append(arc[1])
        if arc[1] not in conflicts:
          conflicts[arc[1]] = [arc[0]]
        else:
          conflicts[arc[1]].append(arc[0])
      return conflicts
    def conflicts(conflicts_list, nodes, node_values): #calculate if we have conflicts and how many with our node_values
      conflict_count = 0
      conflict_nodes = []
      for index, node in enumerate(nodes):
        if node == 'Y': 
          mult = 1
          if index in conflicts_list:
            for n in conflicts_list[index]: # multiplies the neighbors
              mult = mult * node_values[n]
          else:
            mult = 0
          val = mult % 10 # takes the rightmost digit
          if(val != node_values[index]): # We have a conflict if the rightmost digit of the multiple of all its neighbors does not equal the node_values
            conflict_count += 1
            conflict_nodes.append(index)
        if node == 'G':
          sum = 0
          if index in conflicts_list: 
            for n in conflicts_list[index]: #add the neighbors
              sum += node_values[n]
          val = sum % 10 # takes the leftmost digit
          if(val != node_values[index]):  # We have a conflict if the rightmost digit of the sum of all its neighbors does not equal the node_values
            conflict_count += 1
            conflict_nodes.append(index)
        if node == 'B':
          sum = 0
          if index in conflicts_list:
            for n in conflicts_list[index]:
              sum += node_values[n]  # we have a conflict if the leftmost digit of the sum of all its nieghbors does not equal node_values
          while sum >= 10: # takes the leftmost digit by constantly dividing by 10
            sum = sum//10
          if(sum != node_values[index]):
            conflict_count += 1
            conflict_nodes.append(index)
        if node == 'V':
          mult = 1
          if index in conflicts_list:
            for n in conflicts_list[index]:
              mult *= node_values[n] # we have a conflict if the leftmost digit of the product of all its nieghbors does not equal node_values
          else:
            mult = 0
          while mult >= 10: # takes the leftmost digit by constantly dividing by 10
            mult = mult//10
          if(mult != node_values[index]):
            conflict_count += 1
            conflict_nodes.append(index)
      return conflict_count, conflict_nodes # return the conflicting nodes and there count
    def minimize_conflicts(index, node_values, conflicts_list):
      min  = np.inf # start at infinity
      val = node_values[index] # get the current node_value
      for i in range(1,10): #check every possible number at the conflicted node value to see if we can get less conflicts
        node_values[index] = i
        conflict_count, conflict_nodes = conflicts(conflicts_list, nodes, node_values)
        if conflict_count < min: # if we find a lower amount of conflicts
          min = conflict_count
          val = i
      return val # return the lower value
    
    conflicts_list = pot_Conflicts(arcs) # creates our neighbors list for each node_value
    stuck = 0 # a parameter that will stop us from getting stuck on a particular index
    conflict_nodes = [] # keeps track of how many conflicts we have
    for node in nodes:
      node_values.append(np.random.randint(1,9)) # start with some random node_values
    for i in range(0,max_steps): # run through the algorithm for the max amount of steps
      prev_conflict_nodes = conflict_nodes # get the previous conflict_nodes each time
      conflict_count, conflict_nodes = conflicts(conflicts_list, nodes, node_values) # check how many conflicts we have with the current node values
      if(conflict_nodes == prev_conflict_nodes): # check if we have the same conflict_nodes over and over
        stuck = stuck + 1
      else:
        stuck = 0
      if(stuck == len(node_values)): # if we have gone through all conflict_nodes and we still are stuck 
        for i in range(len(node_values)): # get new random node vales
          node_values[i] = np.random.randint(1,9)
        conflict_count, conflict_nodes = conflicts(conflicts_list, nodes, node_values)
        stuck = 0 # reset the stuck value
      if conflict_count == 0: # if we have no conflicts return the node_values
        return node_values
      n = np.random.choice(conflict_nodes) # pick a random conflict node
      val = minimize_conflicts(n, node_values, conflicts_list) # return the value that minimizes conflicts
      node_values[n] = val # and set our node_values index to it
    print("Failed to converge") # if we cant find a solution print failed to converge and return where are node values stopped
    return []

### Test Cases

Below we've included 4 test cases to help you debug your code. Your submitted code will be tested on other cases as well, but if your implementation of the above Min-Conflicts search algorithm is able to solve these problems, you should be good to go.

In [13]:
# test Case 1

nodes = 'YGVRB'
arcs = [(0,1), (0,2), (1,2), (1,3), (1,4), (2,3), (2,4)]
max_steps = 1000

for _ in range(max_steps):
    sol = solve_csp(nodes, arcs, max_steps)
    if sol != []:
        break
        
all_solutions = [[1, 1, 1, 7, 2],[2, 1, 2, 4, 3],[2, 6, 7, 6, 1],[2, 8, 9, 6, 1],
                 [3, 3, 1, 5, 4],[6, 2, 8, 7, 1],[6, 7, 8, 2, 1],[6, 9, 4, 8, 1]]

if sol == []:
    print('No solution')
else:
    if sol in all_solutions:
        print('Solution found:', sol)
    else:
        print('ERROR: False solution found:', sol)

Solution found: [2, 1, 2, 4, 3]


In [5]:
# test Case 2

nodes = 'YVBGR'
arcs = [(0,1), (0,2), (1,3), (2,4)]
max_steps = 1000

for _ in range(max_steps):
    sol = solve_csp(nodes, arcs, max_steps)
    if sol != []:
        print(nodes)
        break
        
all_solutions = [[1, 1, 1, 1, 9], [1, 3, 7, 3, 6], [1, 7, 3, 7, 2], [1, 9, 9, 9, 8]]

if sol == []:
    print('No solution')
else:
    if sol in all_solutions:
        print('Solution found:', sol)
    else:
        print('ERROR: False solution found:', sol)

YVBGR
Solution found: [1, 7, 3, 7, 2]


In [6]:
# test Case 3

nodes = 'VYGBR'
arcs = [(0,1), (1,2), (2,3), (3,4)]
max_steps = 1000

for _ in range(max_steps):
    sol = solve_csp(nodes, arcs, max_steps)
    if sol != []:
        print(nodes)
        break
        
all_solutions = [[2, 2, 1, 9, 8],[3, 3, 1, 8, 7],[4, 4, 1, 7, 6],[5, 5, 1, 6, 5],[5, 5, 3, 8, 5],
                 [6, 6, 1, 5, 4],[7, 7, 1, 4, 3],[8, 8, 1, 3, 2],[8, 8, 6, 8, 2],[9, 9, 1, 2, 1]]

if sol == []:
    print('No solution')
else:
    if sol in all_solutions:
        print('Solution found:', sol)
    else:
        print('ERROR: False solution found:', sol)

VYGBR
Solution found: [8, 8, 1, 3, 2]


In [7]:
# test Case 4

nodes = 'YGVBR'
arcs = [(0,1), (0,2), (1,3), (2,3), (3,4), (1,2)]
max_steps = 1000

for _ in range(max_steps):
    sol = solve_csp(nodes, arcs, max_steps)
    if sol != []:
        print(nodes)
        break
        
all_solutions = [[4, 4, 1, 9, 4],[4, 7, 2, 1, 1],[4, 7, 2, 1, 2],[4, 7, 2, 1, 3],[4, 7, 2, 1, 4],
                 [4, 7, 2, 1, 5],[4, 7, 2, 1, 6],[4, 7, 2, 1, 7],[4, 7, 2, 1, 8],[4, 7, 2, 1, 9],
                 [4, 8, 3, 1, 1],[4, 8, 3, 1, 2],[4, 8, 3, 1, 3],[4, 8, 3, 1, 4],[4, 8, 3, 1, 5],
                 [4, 8, 3, 1, 6],[4, 8, 3, 1, 7],[4, 8, 3, 1, 8],[5, 1, 5, 1, 4],[5, 1, 5, 1, 5],
                 [5, 1, 5, 1, 6],[5, 1, 5, 1, 7],[5, 1, 5, 1, 8],[5, 1, 5, 1, 9]]

if sol == []:
    print('No solution')
else:
    if sol in all_solutions:
        print('Solution found:', sol)
    else:
        print('ERROR: False solution found:', sol)

YGVBR
Solution found: [5, 1, 5, 1, 8]


## Problem 3: The N-Rooks Problem
---

Rooks can move any number of squares horizontally or vertically on a chess board. The n rooks problem is to arrange rooks on an $n\times n$ board in such a way that none of the rooks could bump into another by making any of its possible horizontal or vertical moves. 

For this problem, the variables are each column (labeled 0, 1, ... , $ n-1$), the the domain consists of each possible row (also labeled 0, 1, ... , $ n-1$). In each column we place a rook on row 0, row 1, ... , row $n-1$.

For example, if $n=2$ we have only two solutions to this problem:

    R @
    @ R 
    
or

    @ R
    R @ 
   

### Part 1)
How many possible solutions are there to this CSP, in terms of n?  Also, give a simple proof that your answer is correct.

There are n! possible solutions to the N-rook csp problem. The proof is quite simple consider fixing a rook at a specific position in the first row of the chess board, This means that we can no longer place any more rooks on this diagonal (we also cant place anything more on the first row given the nature of how a rook moves). Therefore, when we move on to the second row we can place our second rook in (n-1) positions, following this pattern we have (n-2) for the third row, (n-3) for the fourth row, etc. Therefore, we can place the rooks in n * (n-1) * (n-2) * (n-3)...1, which is the formal definition of n!.

### Python Library to solve CSPs

One useful Python module for solving these types of problems is called *constraint*. In the next cell you'll see how to load this module into a Jupyter notebook running in Colab. 

We'll use this module to solve the following simple CSP. Suppose our variables are x and y, and the values they are allowed to assume are numbers in the domain {1, 2, ... , 100}, and with constraints be that $x^2=y$ and that $x$ is odd.

In [ ]:
# The following line imports the constraint module into a Jupyter notebook running in Colab.
# (This only needs to be done once per session)

!pip install python-constraint

# Let's load all the functions available in this module
# (This only needs to be done once per session)

from constraint import *

# Now we initialize a new problem and add the two variables.
problem = Problem()
problem.addVariables(["x", "y"], list(range(1, 101)))

# So far there are no constraints. If we solve the problem with no constraints, 
# then every possible combination of x and y values from 1 to 100 are valid solutions.
solutions = problem.getSolutions()

# Total number of solutions
print( len(solutions) )

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for python-constraint: filename=python_constraint-1.4.0-py2.py3-none-any.whl size=24079 sha256=65212894ca3610e214ffe22356e9424ad92e1f98334d050f8a7fbbcce0d8daee
  Stored in directory: /root/.cache/pip/wheels/86/ba/5c/4e9115777de42c6a2e1ca77ef7c9d0d479254c5080341b55c5
Successfully built python-constraint
10000


In [ ]:
# Add the given constraints:
problem.addConstraint(lambda a, b: a**2 == b and (a % 2) == 1, ("x", "y"))

# There are only 5 solutions that satisfy those constraints:
solutions = problem.getSolutions()
print( len(solutions) )

5


In [ ]:
# Here are all 5 solutions:
print(solutions)

[{'x': 9, 'y': 81}, {'x': 7, 'y': 49}, {'x': 5, 'y': 25}, {'x': 3, 'y': 9}, {'x': 1, 'y': 1}]


Notice that the solutions consist of a list of dictionaries, where each dictionary represents a solution. For example, the first solution is {'x': 9, 'y': 81}, since with $x=9$ and $y=81$ it's true that $x^2=y$.

### Part 2) The n-rooks problem revisited

Modify the example before to solve the n-rooks problem.

In [ ]:
from numpy.lib.shape_base import column_stack
### YOUR CODE HERE ###
# reference: https://pypi.org/project/python-constraint/
problem = Problem() #create a problem
n=8 # n value (modifiable)
column = range(n) #we have a n by n board
row = range(n)
problem.addVariables(column, row) #place both in our constraint problem as variables
for col1 in column: 
  for col2 in column: 
    if col1 < col2: # make sure we are not a column already used 
      problem.addConstraint(lambda row1, row2: row1 != row2,
                                  (col1, col2)) # make sure were also not in the same row
solutions = problem.getSolutions()
print(len(solutions))



40320


### Part 3)

*   How many ways are there of arranging $8$ rooks on an $8\times8$ board so that none impede the others?
*   How many ways are there arranging $11$ rooks on an $11\times11$ board so that none impede the others?


1.There are 40,320 different ways to place n=8 rooks or 8!

2.There are 39916800 different ways to place n=11 rooks or 11!
